## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import islice

import pickle as pkl

import sentencepiece as sp

import tkseem as tk
import gensim

In [2]:
df = pd.read_csv('data/apcd_clean.csv')
df['poem_text'] = df['البيت'].astype(str)
df.drop('البيت', inplace=True, axis=1)
df.head()

,poem_text
0,خليلي لا تستعجلا أن تزودا وأن تجمعا شملي وت...
1,فما لبث يوما بسابق مغنم ولا سرعتي يوما بساب...
2,وإن تنظراني اليوم أقض لبانة وتستوجبا منا عل...
3,لعمرك ما نفس بجد رشيدة تؤامرني سرا لأصرم مرثدا
4,وإن ظهرت منه قوارص جمة وأفرع في لومي مرارا ...


## Morphological Tokenizer

In [3]:
morph_tokenizer = tk.MorphologicalTokenizer(vocab_size=500_000)

morph_tokenizer.train()

Training MorphologicalTokenizer ...


In [4]:
text = 'توكلت في رزقي على الله خالقي'

morph_tokenizer.tokenize(text)

['توكلت', 'في', 'رزق', '##ي', 'على', 'الله', 'خالق', '##ي']

In [ ]:
def tokenize_text_morph(text):
    return morph_tokenizer.tokenize(text)

df['morph_t'] = df.poem_text.apply(tokenize_text_morph)
df.head()

In [ ]:
text = 'أق'

morph_tokenizer.tokenize(text)

In [7]:
morph_model = gensim.models.Word2Vec(
    df['morph_t'],
    window=5,
    min_count=4,
    workers=8)

In [8]:
morph_model.wv.most_similar(positive='ثلج')

[('حريق', 0.7407509088516235),
 ('لازورد', 0.7376511693000793),
 ('سبج', 0.7300616502761841),
 ('جلنار', 0.7153370380401611),
 ('زجاج', 0.7149295806884766),
 ('جام', 0.7102290391921997),
 ('لجين', 0.7094608545303345),
 ('ياسمين', 0.7039445638656616),
 ('شرر', 0.7033098936080933),
 ('ذوب', 0.7032054662704468)]

In [7]:
morph_model.save('models/morph_word2vec_model.model')

## Word Tokenizer

In [9]:
word_tokenizer = tk.WordTokenizer()

df['poem_text'].to_csv('data.txt', sep=' ', index=False, header=False)
word_tokenizer.train('data.txt')

Training WordTokenizer ...


In [10]:
def tokenize_text_word(text):
    return word_tokenizer.tokenize(text)

df['word_t'] = df.poem_text.apply(tokenize_text_word)
df.head()

,poem_text,morph_t,word_t
0,خليلي لا تستعجلا أن تزودا وأن تجمعا شملي وت...,"[خليلي, لا, <UNK>, أن, تزودا, و, ##أن, تجمعا, ...","[خليلي, لا, <UNK>, أن, <UNK>, وأن, <UNK>, شملي..."
1,فما لبث يوما بسابق مغنم ولا سرعتي يوما بساب...,"[<UNK>, لبث, يوما, <UNK>, مغنم, <UNK>, سرعت, #...","[فما, <UNK>, يوما, <UNK>, <UNK>, ولا, <UNK>, ي..."
2,وإن تنظراني اليوم أقض لبانة وتستوجبا منا عل...,"[وإن, تنظران, ##ي, ال, ##يوم, أقض, لبانة, <UNK...","[وإن, <UNK>, اليوم, أقض, لبانة, <UNK>, منا, عل..."
3,لعمرك ما نفس بجد رشيدة تؤامرني سرا لأصرم مرثدا,"[لع, ##مر, ##ك, ما, نفس, <UNK>, رشيدة, تؤامر, ...","[لعمرك, ما, نفس, بجد, <UNK>, <UNK>, سرا, <UNK>..."
4,وإن ظهرت منه قوارص جمة وأفرع في لومي مرارا ...,"[وإن, ظهرت, منه, قوارص, جمة, و, ##أفرع, في, لو...","[وإن, ظهرت, منه, <UNK>, جمة, <UNK>, في, لومي, ..."


In [11]:
word_model = gensim.models.Word2Vec(
    df['word_t'],
    window=5,
    min_count=4,
    workers=8)

In [14]:
word_model.wv.most_similar(positive='جمال')

[('بهاء', 0.7381551265716553),
 ('جلال', 0.7211630344390869),
 ('كمال', 0.7165018320083618),
 ('جمالا', 0.6697500944137573),
 ('جماله', 0.6606019735336304),
 ('محيا', 0.6495076417922974),
 ('سناء', 0.6364024877548218),
 ('بهجة', 0.6349396109580994),
 ('ملاحة', 0.6105407476425171),
 ('ضياء', 0.6032314896583557)]

In [12]:
word_model.save('models/word_word2vec_model.model')

## Sentence Piece Tokenization

In [56]:
sp.SentencePieceTrainer.train(
    sentence_iterator=iter(df['poem_text'].values),
    model_prefix='models/sp_poem_model',
    vocab_size=50_000,
    pad_id=0,
    unk_id=1,
    bos_id=2,
    eos_id=3
)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input_format: 
  model_prefix: models/sp_poem_model
  model_type: UNIGRAM
  vocab_size: 50000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 1
  bos_id: 2
  eos_id: 3
  pad_id: 0
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential_pr

In [57]:
spp = sp.SentencePieceProcessor(model_file='models/sp_poem_model.model')

In [58]:
def tokenize_text_sp(text):
    return spp.encode(text, out_type=str)

df['sp_t'] = df.poem_text.apply(tokenize_text_sp)
df.head()

,poem_text,morph_t,word_t,sp_t
0,خليلي لا تستعجلا أن تزودا وأن تجمعا شملي وت...,"[خليلي, لا, <UNK>, أن, تزودا, و, ##أن, تجمعا, ...","[خليلي, لا, <UNK>, أن, <UNK>, وأن, <UNK>, شملي...","[▁خليلي, ▁لا, ▁تستع, جلا, ▁أن, ▁تزود, ا, ▁وأن,..."
1,فما لبث يوما بسابق مغنم ولا سرعتي يوما بساب...,"[<UNK>, لبث, يوما, <UNK>, مغنم, <UNK>, سرعت, #...","[فما, <UNK>, يوما, <UNK>, <UNK>, ولا, <UNK>, ي...","[▁فما, ▁لبث, ▁يوما, ▁بسابق, ▁مغنم, ▁ولا, ▁سرع,..."
2,وإن تنظراني اليوم أقض لبانة وتستوجبا منا عل...,"[وإن, تنظران, ##ي, ال, ##يوم, أقض, لبانة, <UNK...","[وإن, <UNK>, اليوم, أقض, لبانة, <UNK>, منا, عل...","[▁وإن, ▁تنظر, اني, ▁اليوم, ▁أقض, ▁لبان, ة, ▁وت..."
3,لعمرك ما نفس بجد رشيدة تؤامرني سرا لأصرم مرثدا,"[لع, ##مر, ##ك, ما, نفس, <UNK>, رشيدة, تؤامر, ...","[لعمرك, ما, نفس, بجد, <UNK>, <UNK>, سرا, <UNK>...","[▁لعمرك, ▁ما, ▁نفس, ▁بجد, ▁رشيد, ة, ▁تؤام, ر, ..."
4,وإن ظهرت منه قوارص جمة وأفرع في لومي مرارا ...,"[وإن, ظهرت, منه, قوارص, جمة, و, ##أفرع, في, لو...","[وإن, ظهرت, منه, <UNK>, جمة, <UNK>, في, لومي, ...","[▁وإن, ▁ظهرت, ▁منه, ▁قوارص, ▁جمة, ▁وأف, رع, ▁ف..."


In [59]:
sp_model = gensim.models.Word2Vec(
    df['sp_t'],
    window=5,
    min_count=3,
    workers=8)

In [68]:
sp_model.wv.most_similar(positive='أسد')

[('لأم', 0.8726752996444702),
 ('حصن', 0.8706651926040649),
 ('رجل', 0.8677271008491516),
 ('نقع', 0.8672091960906982),
 ('رماح', 0.8636544346809387),
 ('حجار', 0.8592782020568848),
 ('ظفار', 0.8591319918632507),
 ('جادل', 0.8589949607849121),
 ('خطار', 0.8574706315994263),
 ('درع', 0.8571063280105591)]

In [18]:
sp_model.save('models/sp_word2vec_model.model')

## Save Updated Data

In [19]:
df.to_csv('all_poems_tokenized.csv', index=False)